# 1 percent CO2 run for carbon cycle metrics

Could add this to the examples

In [ ]:
import copy

import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import time
import scipy.stats
from tqdm import tqdm

from fair import (
    SpeciesID, Category, Config, Species, RunMode, Scenario, ClimateResponse, RunConfig, CH4LifetimeMethod, FAIR
)
from fair.defaults import species_config_from_default

In [ ]:
species_ids = {
    # Greenhouse gases
    'CO2': SpeciesID('CO2', Category.CO2, run_mode=RunMode.CONCENTRATION),
    'CH4': SpeciesID('CH4', Category.CH4, run_mode=RunMode.CONCENTRATION),
    'N2O': SpeciesID('N2O', Category.N2O, run_mode=RunMode.CONCENTRATION),
}

In [ ]:
scenarios_to_include = ['1pctCO2']

In [ ]:
concentration_species = ['CO2', 'CH4', 'N2O']
species_to_include = concentration_species

In [ ]:
df_conc = pd.read_csv('../data/rcmip/rcmip-concentrations-annual-means-v5-1-0.csv')
df_conc.loc[(df_conc['Scenario']=='1pctCO2') & (df_conc['Variable'].str.endswith("|CO2")),'1850']

In [ ]:
scenarios = []
for iscen, scenario in enumerate(scenarios_to_include):
    list_of_species = []
    for ispec, species in enumerate(concentration_species):
        species_rcmip_name = species.replace("-", "")
        if species == 'NOx_aviation':
            species_rcmip_name = 'NOx|MAGICC Fossil and Industrial|Aircraft'
        elif species == 'CO2_FFI':
            species_rcmip_name = 'CO2|MAGICC Fossil and Industrial'
        elif species == 'CO2_AFOLU':
            species_rcmip_name = 'CO2|MAGICC AFOLU'
        conc_in = df_conc.loc[
            (df_conc['Scenario']==scenario) & (df_conc['Variable'].str.endswith("|"+species_rcmip_name)) & 
            (df_conc['Region']=='World'), '1850':'2000'
        ].interpolate(axis=1).values.squeeze()

        list_of_species.append(Species(species_ids[species], concentration=conc_in))
        #print(species_rcmip_name, emis_in.shape)
        
    scenarios.append(Scenario(scenario, list_of_species))

In [ ]:
valid_all = np.loadtxt('../data/ar6_ensemble_batches/final.csv', dtype=int)#[:1000]
valid_all

In [ ]:
df_cc=pd.read_csv('../data/parameter_sets/carbon_cycle.csv')
df_cr=pd.read_csv('../data/parameter_sets/climate_response.csv')
df_scaling=pd.read_csv('../data/parameter_sets/forcing_scaling.csv')

In [ ]:
species_index = {}
for i in range(len(species_to_include)):
    species_index[species_to_include[i]] = i

In [ ]:
species_index

In [ ]:
co2_pi = df_conc.loc[
            (df_conc['Scenario']==scenario) & (df_conc['Variable'].str.endswith("|CO2")) & 
            (df_conc['Region']=='World'), '1850'
        ].values[0]
ch4_pi = df_conc.loc[
            (df_conc['Scenario']==scenario) & (df_conc['Variable'].str.endswith("|CH4")) & 
            (df_conc['Region']=='World'), '1850'
        ].values[0]
n2o_pi = df_conc.loc[
            (df_conc['Scenario']==scenario) & (df_conc['Variable'].str.endswith("|N2O")) & 
            (df_conc['Region']=='World'), '1850'
        ].values[0]

In [ ]:
seedgen = 1355763
seed = seedgen

calibrated_f4co2_mean = df_cr['F_4xCO2'].mean()

configs = []
for iconf in valid_all:
    config_name = f"{iconf}"

    # Climate response configs
    climate_response = ClimateResponse(
        ocean_heat_capacity = df_cr.loc[iconf, 'c1':'c3'].values.squeeze(),
        ocean_heat_transfer = df_cr.loc[iconf, 'kappa1':'kappa3'].values.squeeze(),
        deep_ocean_efficacy = df_cr.loc[iconf, 'epsilon'],
        gamma_autocorrelation = df_cr.loc[iconf, 'gamma'],
        sigma_eta = df_cr.loc[iconf, 'sigma_eta'],
        sigma_xi = df_cr.loc[iconf, 'sigma_xi'],
        stochastic_run = True,
        seed = seed
    )
    seed = seedgen + 399 * iconf

    # Species configs
    # get defaults
    species_config = [copy.copy(species_config_from_default(species)) for species in species_to_include]
    species_config[0].species_id.run_mode = RunMode.CONCENTRATION
    species_config[1].species_id.run_mode = RunMode.CONCENTRATION
    species_config[2].species_id.run_mode = RunMode.CONCENTRATION
    
    species_config[0].baseline_concentration = co2_pi
    species_config[1].baseline_concentration = ch4_pi
    species_config[2].baseline_concentration = n2o_pi
    
    # CO2 carbon cycle sensitivity
    species_config[0].iirf_0 = df_cc.loc[iconf, 'r0']
    species_config[0].iirf_airborne = df_cc.loc[iconf, 'rA']
    species_config[0].iirf_cumulative = df_cc.loc[iconf, 'rC']
    species_config[0].iirf_temperature = df_cc.loc[iconf, 'rT']

    # overwrite CO2 forcing scaling with correlated parameter from lambda1
    species_config[0].scale = 1 + 0.561*(calibrated_f4co2_mean - df_cr.loc[iconf,'F_4xCO2'])/calibrated_f4co2_mean

    configs.append(Config(config_name, climate_response, species_config))

In [ ]:
fair = FAIR(scenarios, configs)
fair.run(progress=True)

In [ ]:
fig, ax = pl.subplots(1, 1, figsize=(8, 8))
ax.fill_between(
    np.arange(151), 
    np.min(fair.temperature[:, 0, :, 0, 0], axis=1), 
    np.max(fair.temperature[:, 0, :, 0, 0], axis=1),
    color='#000000',
    alpha=0.2,
)
ax.fill_between(
    np.arange(151), 
    np.percentile(fair.temperature[:, 0, :, 0, 0], 5, axis=1), 
    np.percentile(fair.temperature[:, 0, :, 0, 0], 95, axis=1),
    color='#000000',
    alpha=0.2,
)
ax.fill_between(
    np.arange(151), 
    np.percentile(fair.temperature[:, 0, :, 0, 0], 16, axis=1), 
    np.percentile(fair.temperature[:, 0, :, 0, 0], 84, axis=1),
    color='#000000',
    alpha=0.2,
)
ax.plot(
    np.arange(151), 
    np.median(fair.temperature[:, 0, :, 0, 0], axis=1), 
    color='#000000',
)
ax.set_ylim(-1, 10)
ax.set_xlim(0, 150)
ax.axhline(0, color='k', ls=":", lw=0.5)
ax.set_title(scenarios_to_include[0])

In [ ]:
pl.plot(fair.temperature[:, 0, :, 0, 0]);

In [ ]:
np.median(fair.temperature[60:81, 0, :, 0, 0].mean(axis=0))

In [ ]:
fair.airborne_emissions_array[:, 0, 0, 0, 0]

In [ ]:
fair.cumulative_emissions_array[:, 0, 0, 0, 0]

In [ ]:
airborne_fraction_co2 = fair.airborne_emissions_array[:-1, 0, 0, 0, 0] / fair.cumulative_emissions_array[1:, 0, 0, 0, 0]

In [ ]:
pl.plot(airborne_fraction_co2)